<a href="https://colab.research.google.com/github/provbs/LOLai_machine_learning/blob/main/lol_data_getter_%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# API 호출을 위해 requests 모듈을 사용
import requests

#pandas 랑 numpy 사용
import numpy as np
import pandas as pd

#json 사용
import json

#시간지연을 위해 time 사용
import time

# API 에서 공통적으로 사용하는 텍스트를 선언
apiDefault = {
    'region': 'https://kr.api.riotgames.com',  # 한국서버를 대상으로 호출
    'key': 'RGAPI-82c663f0-518b-43b3-b97e-35cd5ed4c6a7',  # API KEY - riot developers portal 에서 가져오기.
    'summonerName': '만판다이야',  # 내 닉네임으로 기본적인 틀 잡음.
}

#################################### 중요!!!!  api 호출시 사용되는 url #########################################
# url = F"{apiDefault['region']}요기에 코드 내용을 적으시요.?api_key={apiDefault['key']}"#
##############################################################################################################

In [ ]:
## 수정본

###################################### 머신러닝 필요 데이터 전처리 ###########################################
############################# 0. 티어별 1000개의 아이디 모아서 dataframe으로 모으기 ###########################

def getUserData(url, tier, tier_num = "", Is_Challenger_Master = False):
    # get method 를 통해 api 호출
    req = requests.get(url)
    #print(req.text)

    #유저들을 모은 "entries" 만 넣기. (앞에 TIER, LEAGUEID, QUEUE, NAME은 제외. 챌린저에서는 단하나의 리그만 존재)
    Users_List = json.loads(req.text)
    
    if Is_Challenger_Master:
        Users_List_entries = pd.DataFrame(Users_List["entries"])

        tier_list = Users_List["tier"]

        Users_List_tiers =[]
        for i in range(len(Users_List_entries)):
            Users_List_tiers.append(tier_list)
        Users_List_tiers = pd.DataFrame(Users_List_tiers, columns=['tier'])
        #display(Users_List_tiers)

        Users_List = pd.concat([Users_List_entries,Users_List_tiers],axis=1)

    #유저들 리스트 dataframe으로 넣고 정렬
    Users_List_DF = pd.DataFrame(Users_List).sort_values(by='leaguePoints' ,ascending=False).reset_index()
    #display(Users_List_DF)
    
    print(F"{tier} {tier_num} finished...!\n")
    return Users_List_DF

In [ ]:
################################# 1. 티어별 매치리스트 모으기 ################################################
############################### 모든 티어의 매치들을 구한후 겹치는 매치는 제거한다 #############################

def getMatchData(Total_Users_List_DF, tier, tier_num = 'I', division_by = 1):
    Users_List_DF = Total_Users_List_DF[(Total_Users_List_DF['tier'] == tier) & (Total_Users_List_DF['rank'] == tier_num)]
    #유저 리스트 데이터 프레임에서 summonerName만 추출해서 dataframe형식으로 저장한다.
    Users_List_DF_summonerName = pd.DataFrame(Users_List_DF['summonerName'])
    display(Users_List_DF_summonerName)

    #초기화
    Users_Puuids = []
    Users_MatchID = []

    #for문을 통해 puuids리스트를 얻기.
    user_count = int(len(Users_List_DF_summonerName)/division_by)
    for i in range(1):
        while True:
            try:
                url = F"{apiDefault['region']}/lol/summoner/v4/summoners/by-name/{Users_List_DF_summonerName.loc[i].to_numpy()[0]}?api_key={apiDefault['key']}"
                # get method 를 통해 api 호출
                req = requests.get(url)
                Users_Puuids_temp = json.loads(req.text)
                print("User Number: ", i, "/", user_count, "\n")
                Users_Puuids.append(Users_Puuids_temp['puuid'])
                break
            except KeyError:
                print("Summonor not found, ignore and skip to next summoner...")
                break
        time.sleep(1)

    print("finished...\n")
    #display(Users_Puuids)

    #for문을 통해 match id 리스트를 얻기
    for n in range(len(Users_Puuids)):
        #유저들의 puuid를 통해 match id를 모은다.
        url = F"{'https://asia.api.riotgames.com'}/lol/match/v5/matches/by-puuid/{Users_Puuids[n]}/ids?type=ranked&api_key={apiDefault['key']}"

        # get method 를 통해 api 호출
        print("Get Match id - User Number: ", n, "/",len(Users_Puuids), "\n")
        req = requests.get(url)
        Users_MatchID_temp = json.loads(req.text)
        Users_MatchID.append(Users_MatchID_temp)
        #print(Users_MatchID_temp)
        time.sleep(1.1)

    #display(Users_MatchID)
    print("finished...!\n")
    return Users_Puuids, Users_MatchID

In [ ]:

Full_Tier = ["CHALLENGER","GRANDMASTER","MASTER", "DIAMOND", "PLATINUM", "GOLD", "SILVER", "BRONZE", "IRON"]
Challenger_Master = ["challengerleagues", "grandmasterleagues", "masterleagues"]
Diamond_Iron = ["DIAMOND", "PLATINUM", "GOLD", "SILVER", "BRONZE", "IRON"]
Diamond_Iron_tIers = ["I", "II","III", "IV"]


##### getUserData #####
full_User_Data = pd.DataFrame()
#[CHALLENGER ~ MASTER]
for tier in Challenger_Master:
    url = F"{apiDefault['region']}/lol/league/v4/{tier}/by-queue/{'RANKED_SOLO_5x5'}?api_key={apiDefault['key']}"
    full_User_Data = full_User_Data.append(getUserData(url, tier, Is_Challenger_Master = True))

#[DIAMOND ~ IRON]
for tier in Diamond_Iron:
    for tier_num in Diamond_Iron_tIers:
        url = F"{apiDefault['region']}/lol/league/v4/entries/{'RANKED_SOLO_5x5'}/{tier}/{tier_num}?page=1&api_key={apiDefault['key']}"
        full_User_Data = full_User_Data.append(getUserData(url, tier, tier_num, Is_Challenger_Master = False ))

full_User_Data.to_csv("full_User_Data.csv", mode='w', encoding="utf-8-sig")
print("CSV file saved...!")
##### getUserData #####

##### getMatchData #####
full_Match_ID = pd.DataFrame()
division_by = 1
for tier in Full_Tier:
    if tier not in Diamond_Iron:
        if tier in "GRANDMASTER":
            division_by = 5
        Users_Puuids, Users_MatchID = getMatchData(full_User_Data, tier, tier_num = 'I',division_by=division_by)
        Users_MatchID_save = sum(Users_MatchID, [])
        Users_MatchID_save = pd.DataFrame(Users_MatchID_save, columns=['MatchID'])
        #display(League_Users_MatchID_save)
        full_Match_ID = full_Match_ID.append(Users_MatchID_save)
        Users_MatchID_save.to_csv(f"{tier}_League_Users_MatchID.csv", mode='w', encoding="utf-8-sig")
        print(f"{tier} DONE...")

    else:
        division_by = 2
        for tier_num in Diamond_Iron_tIers:
            Users_Puuids, Users_MatchID = getMatchData(full_User_Data, tier, tier_num, division_by)
            Users_MatchID_save = sum(Users_MatchID, []) #복사본 만들기
            Users_MatchID_save = pd.DataFrame(Users_MatchID_save, columns=['MatchID'])
            #display(League_Users_MatchID_save)
            full_Match_ID = full_Match_ID.append(Users_MatchID_save)
            Users_MatchID_save.to_csv(f"{tier}_{tier_num}_League_Users_MatchID.csv", mode='w', encoding="utf-8-sig")
            print(f"{tier} {tier_num} DONE...")

print("Originally : ",len(full_Match_ID), " matches.\n")

full_Match_ID_No_Duplicate = full_Match_ID.drop_duplicates(['MatchID'])

print("After Removing duplicates : ",len(full_Match_ID_No_Duplicate), " matches.\n")
##### getMatchData #####

full_Match_ID_No_Duplicate_divided = np.array_split(full_Match_ID_No_Duplicate, 4)
for index, match_chunk in enumerate(full_Match_ID_No_Duplicate_divided):
    chunk.to_csv(f"Merge_Match_part_{index}_no_duplicates.csv", mode='w')
    print("Divided match files saved as csv..! Match Numbers: ", len(match_chunk))

,summonerName
0,이 차가 식기전에
1,fragiIe
2,디알엑스 태윤
3,Porte de I Enfer
4,칼과 창 방패
...,...
204,IRIDESCNT
205,펭귄빙산추락
206,ynnkaa
207,99도의 사나이


User Number:  0 / 209 

finished...

Get Match id - User Number:  0 / 1 

finished...!

CHALLENGER DONE...


,summonerName
0,SwordArt77777
1,sanspareil2
2,Kishimiyu
3,xiangcai
4,땀비몬
...,...
566,노런색
567,마법공학 상자
568,어쩔카정
569,super secret


User Number:  0 / 114 

finished...

Get Match id - User Number:  0 / 1 

finished...!

GRANDMASTER DONE...


,summonerName
0,이품진
1,Luckyy game
2,zyf
3,FATE
4,흠 산
...,...
1675,불곰국가
1676,속도제어
1677,수유 Madlife
1678,정신나갈거같애


...

User Number:  0 / 102 

finished...

Get Match id - User Number:  0 / 1 

finished...!

IRON II DONE...


,summonerName
0,민호야너나잘해
1,dong112
2,풍호동골동품
3,좀만한 동식이
4,lilfdfhfgfmbnfdf
...,...
200,yongkyun
201,뭐라하면노갱
202,롤에서 슈팅게임
203,튼튼한기둥맛볼래


User Number:  0 / 102 

finished...

Get Match id - User Number:  0 / 1 

finished...!

IRON III DONE...


,summonerName
0,공주님유하
1,그러고산다
2,hyunw love
3,사미라 초보 봇
4,목포불주먹상워니
...,...
200,바투자키
201,우리 강아지 여름
202,심해탐방열차
203,서울시자취남


User Number:  0 / 102 

finished...

Get Match id - User Number:  0 / 1 

finished...!

IRON IV DONE...
Originally :  534  matches.

After Removing duplicates :  534  matches.



In [ ]:
##################################### 2.          매치 세부정보 더하기 ##########################################
################### 1.에서 구한 매치리스트를 토대로 랭킹이 포함된 매치리스트를 더해서 큰 dataframe을 얻는다. #########

#for 문을 통해 실제 매치 정보 모으기
## 1/4 씩 나눠서 계산하기.
def get_match_info(match_chunk):
    League_Match_Info_chunk = pd.DataFrame()
    while True:
        try:
            match_chunk = match_chunk.reset_index()
            break
        except:
            print("index already reset...")
            break

    match_chunk_len = int(len(match_chunk)/30)
    for i in range(match_chunk_len):
        try:
            print("Match Number: ", i, "/",match_chunk_len, "\n")
            print("Working on this match...\n")
            url = F"{'https://asia.api.riotgames.com'}/lol/match/v5/matches/{match_chunk['MatchID'].loc[i]}?api_key={apiDefault['key']}"
            time.sleep(1)
            # get method 를 통해 api 호출
            req = requests.get(url)
            #print(req.text)

            #해당 매치 세부정보 중 PARTICIPANTS만 모은다.
            League_Match_Info = json.loads(req.text)
            League_Match_Info = League_Match_Info["info"]["participants"]

            #해당 매치 세부정보를 데이터프레임으로 넣어서 보기.
            League_Match_Info_DF = pd.DataFrame(League_Match_Info)
            League_Match_Info_chunk = League_Match_Info_chunk.append(League_Match_Info_DF)
        except Exception as e:
            print("error: ", e)
            print("Match not recorded.", e)
    return League_Match_Info_chunk

for index, match_chunk in enumerate(full_Match_ID_No_Duplicate_quartered):
    #한 iteration이 길어서 key가 만료되므로, 우선 if(index == ?)로 제어 (key 유효기간이 길어지면 삭제)
    if (index==0):
        League_Match_Info_chunk = pd.DataFrame(get_match_info(match_chunk))
        try:
            League_Match_Info_chunk = League_Match_Info_chunk.reset_index()
        except:
            print("index already reset...")
        League_Match_Info_chunk = League_Match_Info_chunk.drop("index", axis=1)
        League_Match_Info_chunk.to_csv(f'FINAL_match_to_rank_p{index}_unprocessed.csv', mode='w', encoding="utf-8-sig")
        print(f"League_Match_Info_chunk_{index} finished...")
# display(League_Match_Info_chunk)

Match Number:  0 / 4 

Working on this match...

Match Number:  1 / 4 

Working on this match...

Match Number:  2 / 4 

Working on this match...

Match Number:  3 / 4 

Working on this match...

League_Match_Info_chunk_0 finished...
